# Advanced Deep Learning Models for Weather Regulation Prediction

This notebook demonstrates the advanced deep learning capabilities of the Weather Regulation Prediction System, including:

1. **LSTM Networks** - For sequential weather pattern analysis
2. **Transformer Models** - State-of-the-art attention-based architectures
3. **CNN Models** - For spatial weather pattern recognition
4. **Attention-LSTM** - Combining LSTM with attention mechanisms
5. **Autoencoder** - For unsupervised feature learning
6. **Ensemble Methods** - Combining multiple deep learning models

## Prerequisites

Ensure you have TensorFlow installed:
```bash
pip install tensorflow
```

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Check TensorFlow availability
try:
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
    print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("TensorFlow not available. Some models will be skipped.")
    TENSORFLOW_AVAILABLE = False

# Set random seeds for reproducibility
np.random.seed(42)
if TENSORFLOW_AVAILABLE:
    tf.random.set_seed(42)

print("Setup complete!")

<cell_type>markdown</cell_type>## 1. Working with Balanced Dataset for Deep Learning

Deep learning models, while powerful, are particularly sensitive to class imbalance. When regulations occur less than 10% of the time (typical in real aviation data), deep learning models often learn to simply predict "no regulation" most of the time, failing to capture the complex patterns that lead to actual regulations.

### Benefits of Balanced Datasets for Deep Learning:

1. **Better Feature Learning**: Neural networks can learn meaningful representations of both classes
2. **Improved Gradient Flow**: Equal class representation helps gradients flow properly during backpropagation
3. **Enhanced Pattern Recognition**: Models learn to recognize subtle patterns that distinguish regulation events
4. **Reduced Bias**: Prevents models from being overly conservative in predictions
5. **Better Calibration**: Prediction probabilities are more reliable and meaningful

Let's load the balanced dataset specifically prepared for this advanced analysis.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Check TensorFlow availability
try:
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
    print(f"GPU available: {tf.config.list_physical_devices('GPU')}")
    TENSORFLOW_AVAILABLE = True
except ImportError:
    print("TensorFlow not available. Some models will be skipped.")
    TENSORFLOW_AVAILABLE = False

# Set random seeds for reproducibility
np.random.seed(42)
if TENSORFLOW_AVAILABLE:
    tf.random.set_seed(42)

# Load balanced dataset for deep learning
import os

balanced_data_path = '../data/balanced_weather_data.csv'

if os.path.exists(balanced_data_path):
    print("\n🎯 Loading balanced dataset for deep learning...")
    balanced_data = pd.read_csv(balanced_data_path)
    
    # Convert timestamp to datetime
    balanced_data['timestamp'] = pd.to_datetime(balanced_data['timestamp'])
    
    print(f"✅ Balanced dataset loaded successfully!")
    print(f"   • Dataset shape: {balanced_data.shape}")
    print(f"   • Regulation distribution: {balanced_data['has_regulation'].value_counts().to_dict()}")
    print(f"   • Regulation rate: {balanced_data['has_regulation'].mean():.1%}")
    print(f"   • Perfect balance for deep learning! 🚀")
    
    use_balanced = True
    weather_data = balanced_data
else:
    print("⚠️  Balanced dataset not found. Generating synthetic data for demonstration...")
    use_balanced = False

print("\nSetup complete!")

In [ ]:
<cell_type>markdown</cell_type>## 2. Data Preparation for Deep Learning (Alternative)

If the balanced dataset is not available, we'll generate more complex synthetic data. However, using the balanced dataset is strongly recommended for production deep learning models.

In [ ]:
# Generate more complex time series data for deep learning (only if balanced dataset not available)
if not use_balanced:
    def generate_complex_weather_sequences(n_samples=2000, sequence_length=24):
        """Generate complex weather time series with multiple patterns"""
        
        # Time features
        timestamps = pd.date_range('2023-01-01', periods=n_samples, freq='1H')
        hour_of_day = timestamps.hour
        day_of_year = timestamps.dayofyear
        
        # Multiple weather patterns
        # 1. Diurnal temperature cycle
        temp_diurnal = 15 + 10 * np.sin(2 * np.pi * hour_of_day / 24)
        
        # 2. Seasonal temperature cycle
        temp_seasonal = 8 * np.sin(2 * np.pi * day_of_year / 365)
        
        # 3. Weather system movements (3-day cycles)
        temp_systems = 5 * np.sin(2 * np.pi * np.arange(n_samples) / (3*24))
        
        # Combine temperature patterns with noise
        temperature = temp_diurnal + temp_seasonal + temp_systems + np.random.normal(0, 2, n_samples)
        
        # Pressure anticorrelated with temperature systems
        pressure = 1013 - 0.5 * temp_systems + np.random.normal(0, 8, n_samples)
        
        # Wind patterns
        wind_base = 8 + 5 * np.sin(2 * np.pi * hour_of_day / 24 + np.pi/3)
        wind_speed = np.abs(wind_base + 3 * np.sin(2 * np.pi * np.arange(n_samples) / (2*24)) + 
                            np.random.normal(0, 2, n_samples))
        
        wind_direction = (180 + 60 * np.sin(2 * np.pi * np.arange(n_samples) / (4*24)) + 
                         np.random.normal(0, 30, n_samples)) % 360
        
        # Visibility affected by humidity and weather conditions
        humidity = 50 + 30 * np.sin(2 * np.pi * hour_of_day / 24 + np.pi) + np.random.normal(0, 10, n_samples)
        humidity = np.clip(humidity, 10, 100)
        
        # Visibility decreases with high humidity and adverse weather
        visibility_base = 15000 - 100 * humidity
        visibility = np.clip(visibility_base + np.random.normal(0, 2000, n_samples), 1000, 20000)
        
        # Cloud cover patterns
        cloud_cover = np.clip(humidity / 2 + np.random.normal(0, 20, n_samples), 0, 100)
        
        # Create DataFrame
        data = pd.DataFrame({
            'timestamp': timestamps,
            'temperature': temperature,
            'pressure': pressure,
            'wind_speed': wind_speed,
            'wind_direction': wind_direction,
            'visibility': visibility,
            'humidity': humidity,
            'cloud_cover': cloud_cover,
            'hour': hour_of_day,
            'day_of_year': day_of_year
        })
        
        return data

    # Generate data
    weather_data = generate_complex_weather_sequences(n_samples=2000)
    print(f"Generated weather data shape: {weather_data.shape}")
    print("\nWeather data sample:")
    print(weather_data.head())
else:
    print("✅ Using balanced dataset - skipping synthetic data generation")

## 2. Sequence Data Preparation

Deep learning models often work with sequences. Let's prepare our data for sequential models.

In [ ]:
def create_sequences(data, sequence_length=24, target_col='has_regulation'):
    """Create sequences for deep learning models"""
    
    # Select features for sequences
    feature_cols = ['temperature', 'pressure', 'wind_speed', 'wind_direction', 
                   'visibility', 'humidity', 'cloud_cover', 'weather_severity']
    
    X_sequences = []
    y_sequences = []
    
    for i in range(sequence_length, len(data)):
        # Get sequence of features
        sequence = data[feature_cols].iloc[i-sequence_length:i].values
        target = data[target_col].iloc[i]
        
        X_sequences.append(sequence)
        y_sequences.append(target)
    
    return np.array(X_sequences), np.array(y_sequences), feature_cols

# Create sequences
sequence_length = 24  # 24 hours of history
X_sequences, y_sequences, feature_names = create_sequences(weather_data, sequence_length)

print(f"Sequence data shape: {X_sequences.shape}")
print(f"Target shape: {y_sequences.shape}")
print(f"Features: {feature_names}")
print(f"Regulation rate in sequences: {y_sequences.mean():.1%}")

# Split data for deep learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split sequences
X_train_seq, X_temp_seq, y_train_seq, y_temp_seq = train_test_split(
    X_sequences, y_sequences, test_size=0.4, random_state=42, stratify=y_sequences
)
X_val_seq, X_test_seq, y_val_seq, y_test_seq = train_test_split(
    X_temp_seq, y_temp_seq, test_size=0.5, random_state=42, stratify=y_temp_seq
)

# Scale the sequences
scaler = StandardScaler()

# Reshape for scaling
X_train_reshaped = X_train_seq.reshape(-1, X_train_seq.shape[-1])
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train_seq_scaled = X_train_scaled.reshape(X_train_seq.shape)

X_val_reshaped = X_val_seq.reshape(-1, X_val_seq.shape[-1])
X_val_scaled = scaler.transform(X_val_reshaped)
X_val_seq_scaled = X_val_scaled.reshape(X_val_seq.shape)

X_test_reshaped = X_test_seq.reshape(-1, X_test_seq.shape[-1])
X_test_scaled = scaler.transform(X_test_reshaped)
X_test_seq_scaled = X_test_scaled.reshape(X_test_seq.shape)

print(f"\nTraining sequences: {X_train_seq_scaled.shape}")
print(f"Validation sequences: {X_val_seq_scaled.shape}")
print(f"Test sequences: {X_test_seq_scaled.shape}")

def create_sequences(data, sequence_length=24, target_col='has_regulation'):
    """Create sequences for deep learning models"""
    
    # Select features for sequences based on available columns
    if use_balanced:
        # For balanced dataset, select appropriate numerical features
        feature_cols = [col for col in data.columns 
                       if col not in ['timestamp', 'airport', 'has_regulation', 'weather_code', 'flight_category', 'regulation_type'] 
                       and data[col].dtype in ['float64', 'int64']]
        # Ensure we have key weather features
        key_features = ['temperature', 'pressure', 'wind_speed', 'wind_direction', 'visibility', 'humidity']
        feature_cols = [col for col in key_features if col in feature_cols] + \
                      [col for col in feature_cols if col not in key_features][:8]  # Limit to avoid too many features
        feature_cols = feature_cols[:8]  # Keep reasonable number of features
    else:
        # For synthetic data
        feature_cols = ['temperature', 'pressure', 'wind_speed', 'wind_direction', 
                       'visibility', 'humidity', 'cloud_cover', 'weather_severity']
    
    print(f"Using features for sequences: {feature_cols}")
    
    X_sequences = []
    y_sequences = []
    
    for i in range(sequence_length, len(data)):
        # Get sequence of features
        sequence = data[feature_cols].iloc[i-sequence_length:i].values
        target = data[target_col].iloc[i]
        
        X_sequences.append(sequence)
        y_sequences.append(target)
    
    return np.array(X_sequences), np.array(y_sequences), feature_cols

# Create sequences
sequence_length = 24  # 24 hours of history (or 24 time steps for balanced data)
X_sequences, y_sequences, feature_names = create_sequences(weather_data, sequence_length)

print(f"Sequence data shape: {X_sequences.shape}")
print(f"Target shape: {y_sequences.shape}")
print(f"Features: {feature_names}")
print(f"Regulation rate in sequences: {y_sequences.mean():.1%}")

# Display class balance
if use_balanced:
    print(f"✅ Class distribution in sequences:")
    unique, counts = np.unique(y_sequences, return_counts=True)
    for cls, count in zip(unique, counts):
        print(f"   Class {cls}: {count} samples ({count/len(y_sequences)*100:.1f}%)")

In [ ]:
if TENSORFLOW_AVAILABLE:
    from models.lstm import LSTMModel
    from config import LSTMConfig
    from training.trainer import Trainer
    
    # Configure LSTM model
    lstm_config = LSTMConfig(
        units=64,
        dropout=0.3,
        recurrent_dropout=0.3,
        bidirectional=True,
        batch_size=32,
        epochs=50,
        sequence_length=sequence_length,
        learning_rate=0.001,
        early_stopping_patience=10
    )
    
    print("Training LSTM model...")
    print(f"Configuration: {lstm_config.__dict__}")
    
    # Create and train model
    lstm_model = LSTMModel(lstm_config)
    trainer = Trainer()
    
    lstm_results = trainer.train_model(
        model=lstm_model,
        X_train=X_train_seq_scaled,
        y_train=y_train_seq,
        X_val=X_val_seq_scaled,
        y_val=y_val_seq,
        model_name="lstm_weather"
    )
    
    print(f"\nLSTM Training Results:")
    print(f"- Accuracy: {lstm_results['accuracy']:.3f}")
    print(f"- Precision: {lstm_results['precision']:.3f}")
    print(f"- Recall: {lstm_results['recall']:.3f}")
    print(f"- F1 Score: {lstm_results['f1_score']:.3f}")
    print(f"- Training Time: {lstm_results['training_time']:.2f}s")
    
else:
    print("TensorFlow not available, skipping LSTM training")
    lstm_model = None
    lstm_results = None

In [ ]:
# Visualize LSTM training history if available
if TENSORFLOW_AVAILABLE and lstm_model is not None:
    # Get training history
    if hasattr(lstm_model, 'training_history') and lstm_model.training_history:
        history = lstm_model.training_history
        
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss curves
        if 'loss' in history:
            axes[0].plot(history['loss'], label='Training Loss')
            if 'val_loss' in history:
                axes[0].plot(history['val_loss'], label='Validation Loss')
            axes[0].set_title('LSTM Training Loss')
            axes[0].set_xlabel('Epoch')
            axes[0].set_ylabel('Loss')
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
        
        # Accuracy curves
        if 'accuracy' in history:
            axes[1].plot(history['accuracy'], label='Training Accuracy')
            if 'val_accuracy' in history:
                axes[1].plot(history['val_accuracy'], label='Validation Accuracy')
            axes[1].set_title('LSTM Training Accuracy')
            axes[1].set_xlabel('Epoch')
            axes[1].set_ylabel('Accuracy')
            axes[1].legend()
            axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    else:
        print("Training history not available")
        
    # Evaluate on test set
    lstm_test_metrics = lstm_model.evaluate(X_test_seq_scaled, y_test_seq)
    print(f"\nLSTM Test Results:")
    print(f"- Accuracy: {lstm_test_metrics.accuracy:.3f}")
    print(f"- Precision: {lstm_test_metrics.precision:.3f}")
    print(f"- Recall: {lstm_test_metrics.recall:.3f}")
    print(f"- F1 Score: {lstm_test_metrics.f1_score:.3f}")
    print(f"- AUC-ROC: {lstm_test_metrics.auc_roc:.3f}")
else:
    print("LSTM model not available for evaluation")

## 4. Transformer Model

Now let's train a Transformer model for weather sequence analysis.

In [ ]:
if TENSORFLOW_AVAILABLE:
    from models.transformer import TransformerModel
    from config import TransformerConfig
    
    # Configure Transformer model
    transformer_config = TransformerConfig(
        d_model=128,
        num_heads=8,
        num_layers=4,
        dropout=0.1,
        sequence_length=sequence_length,
        batch_size=32,
        epochs=30,  # Fewer epochs for demo
        learning_rate=0.0001,
        warmup_steps=1000
    )
    
    print("Training Transformer model...")
    print(f"Configuration: {transformer_config.__dict__}")
    
    # Create and train model
    transformer_model = TransformerModel(transformer_config)
    
    transformer_results = trainer.train_model(
        model=transformer_model,
        X_train=X_train_seq_scaled,
        y_train=y_train_seq,
        X_val=X_val_seq_scaled,
        y_val=y_val_seq,
        model_name="transformer_weather"
    )
    
    print(f"\nTransformer Training Results:")
    print(f"- Accuracy: {transformer_results['accuracy']:.3f}")
    print(f"- Precision: {transformer_results['precision']:.3f}")
    print(f"- Recall: {transformer_results['recall']:.3f}")
    print(f"- F1 Score: {transformer_results['f1_score']:.3f}")
    print(f"- Training Time: {transformer_results['training_time']:.2f}s")
    
    # Test evaluation
    transformer_test_metrics = transformer_model.evaluate(X_test_seq_scaled, y_test_seq)
    print(f"\nTransformer Test Results:")
    print(f"- Accuracy: {transformer_test_metrics.accuracy:.3f}")
    print(f"- Precision: {transformer_test_metrics.precision:.3f}")
    print(f"- Recall: {transformer_test_metrics.recall:.3f}")
    print(f"- F1 Score: {transformer_test_metrics.f1_score:.3f}")
    print(f"- AUC-ROC: {transformer_test_metrics.auc_roc:.3f}")
    
else:
    print("TensorFlow not available, skipping Transformer training")
    transformer_model = None
    transformer_results = None

## 5. Attention-LSTM Model

Let's try an LSTM with attention mechanism for improved performance.

In [ ]:
if TENSORFLOW_AVAILABLE:
    from models.attention_lstm import AttentionLSTMModel
    from config import LSTMConfig  # Reuse LSTM config for attention model
    
    # Configure Attention-LSTM model
    attention_lstm_config = LSTMConfig(
        units=64,
        dropout=0.3,
        recurrent_dropout=0.3,
        batch_size=32,
        epochs=40,
        sequence_length=sequence_length,
        learning_rate=0.001
    )
    
    print("Training Attention-LSTM model...")
    
    # Create and train model
    attention_lstm_model = AttentionLSTMModel(attention_lstm_config)
    
    attention_lstm_results = trainer.train_model(
        model=attention_lstm_model,
        X_train=X_train_seq_scaled,
        y_train=y_train_seq,
        X_val=X_val_seq_scaled,
        y_val=y_val_seq,
        model_name="attention_lstm_weather"
    )
    
    print(f"\nAttention-LSTM Training Results:")
    print(f"- Accuracy: {attention_lstm_results['accuracy']:.3f}")
    print(f"- Precision: {attention_lstm_results['precision']:.3f}")
    print(f"- Recall: {attention_lstm_results['recall']:.3f}")
    print(f"- F1 Score: {attention_lstm_results['f1_score']:.3f}")
    print(f"- Training Time: {attention_lstm_results['training_time']:.2f}s")
    
    # Test evaluation
    attention_lstm_test_metrics = attention_lstm_model.evaluate(X_test_seq_scaled, y_test_seq)
    print(f"\nAttention-LSTM Test Results:")
    print(f"- Accuracy: {attention_lstm_test_metrics.accuracy:.3f}")
    print(f"- Precision: {attention_lstm_test_metrics.precision:.3f}")
    print(f"- Recall: {attention_lstm_test_metrics.recall:.3f}")
    print(f"- F1 Score: {attention_lstm_test_metrics.f1_score:.3f}")
    print(f"- AUC-ROC: {attention_lstm_test_metrics.auc_roc:.3f}")
    
else:
    print("TensorFlow not available, skipping Attention-LSTM training")
    attention_lstm_model = None
    attention_lstm_results = None

## 6. CNN Model for Spatial Patterns

CNNs can capture spatial patterns in weather data when reshaped appropriately.

In [ ]:
if TENSORFLOW_AVAILABLE:
    from models.cnn import CNNModel
    from config import CNNConfig
    
    # Reshape sequence data for CNN (treat sequence as 2D image)
    # For CNN: (samples, height, width, channels)
    # We'll reshape to treat time as height and features as width
    def reshape_for_cnn(X_seq):
        # X_seq shape: (samples, time_steps, features)
        # Reshape to: (samples, time_steps, features, 1)
        return np.expand_dims(X_seq, axis=-1)
    
    X_train_cnn = reshape_for_cnn(X_train_seq_scaled)
    X_val_cnn = reshape_for_cnn(X_val_seq_scaled)
    X_test_cnn = reshape_for_cnn(X_test_seq_scaled)
    
    print(f"CNN input shape: {X_train_cnn.shape}")
    
    # Configure CNN model
    cnn_config = CNNConfig(
        filters=[32, 64, 128],
        kernel_sizes=[3, 3, 3],
        pool_sizes=[2, 2, 2],
        dropout=0.3,
        batch_size=32,
        epochs=30,
        learning_rate=0.001
    )
    
    print("Training CNN model...")
    
    # Create and train model
    cnn_model = CNNModel(cnn_config)
    
    cnn_results = trainer.train_model(
        model=cnn_model,
        X_train=X_train_cnn,
        y_train=y_train_seq,
        X_val=X_val_cnn,
        y_val=y_val_seq,
        model_name="cnn_weather"
    )
    
    print(f"\nCNN Training Results:")
    print(f"- Accuracy: {cnn_results['accuracy']:.3f}")
    print(f"- Precision: {cnn_results['precision']:.3f}")
    print(f"- Recall: {cnn_results['recall']:.3f}")
    print(f"- F1 Score: {cnn_results['f1_score']:.3f}")
    print(f"- Training Time: {cnn_results['training_time']:.2f}s")
    
    # Test evaluation
    cnn_test_metrics = cnn_model.evaluate(X_test_cnn, y_test_seq)
    print(f"\nCNN Test Results:")
    print(f"- Accuracy: {cnn_test_metrics.accuracy:.3f}")
    print(f"- Precision: {cnn_test_metrics.precision:.3f}")
    print(f"- Recall: {cnn_test_metrics.recall:.3f}")
    print(f"- F1 Score: {cnn_test_metrics.f1_score:.3f}")
    print(f"- AUC-ROC: {cnn_test_metrics.auc_roc:.3f}")
    
else:
    print("TensorFlow not available, skipping CNN training")
    cnn_model = None
    cnn_results = None

## 7. Autoencoder for Feature Learning

Let's use an autoencoder to learn compressed representations of weather patterns.

In [ ]:
if TENSORFLOW_AVAILABLE:
    from models.autoencoder import AutoencoderModel
    from config import AutoencoderConfig
    
    # Flatten sequences for autoencoder
    X_train_flat = X_train_seq_scaled.reshape(X_train_seq_scaled.shape[0], -1)
    X_val_flat = X_val_seq_scaled.reshape(X_val_seq_scaled.shape[0], -1)
    X_test_flat = X_test_seq_scaled.reshape(X_test_seq_scaled.shape[0], -1)
    
    print(f"Flattened input shape: {X_train_flat.shape}")
    
    # Configure Autoencoder
    autoencoder_config = AutoencoderConfig(
        encoding_dims=[128, 64, 32],  # Compress to 32 dimensions
        dropout=0.2,
        batch_size=32,
        pretrain_epochs=20,  # Unsupervised pretraining
        epochs=30,  # Supervised fine-tuning
        learning_rate=0.001
    )
    
    print("Training Autoencoder model...")
    
    # Create and train model
    autoencoder_model = AutoencoderModel(autoencoder_config)
    
    autoencoder_results = trainer.train_model(
        model=autoencoder_model,
        X_train=X_train_flat,
        y_train=y_train_seq,
        X_val=X_val_flat,
        y_val=y_val_seq,
        model_name="autoencoder_weather"
    )
    
    print(f"\nAutoencoder Training Results:")
    print(f"- Accuracy: {autoencoder_results['accuracy']:.3f}")
    print(f"- Precision: {autoencoder_results['precision']:.3f}")
    print(f"- Recall: {autoencoder_results['recall']:.3f}")
    print(f"- F1 Score: {autoencoder_results['f1_score']:.3f}")
    print(f"- Training Time: {autoencoder_results['training_time']:.2f}s")
    
    # Test evaluation
    autoencoder_test_metrics = autoencoder_model.evaluate(X_test_flat, y_test_seq)
    print(f"\nAutoencoder Test Results:")
    print(f"- Accuracy: {autoencoder_test_metrics.accuracy:.3f}")
    print(f"- Precision: {autoencoder_test_metrics.precision:.3f}")
    print(f"- Recall: {autoencoder_test_metrics.recall:.3f}")
    print(f"- F1 Score: {autoencoder_test_metrics.f1_score:.3f}")
    print(f"- AUC-ROC: {autoencoder_test_metrics.auc_roc:.3f}")
    
    # Extract learned features
    encoded_features = autoencoder_model.extract_features(X_test_flat)
    print(f"\nLearned feature dimensions: {encoded_features.shape[1]}")
    
else:
    print("TensorFlow not available, skipping Autoencoder training")
    autoencoder_model = None
    autoencoder_results = None

## 8. Ensemble of Deep Learning Models

Let's create an ensemble combining our best deep learning models.

In [ ]:
if TENSORFLOW_AVAILABLE:
    from models.ensemble import EnsembleModel
    from config import EnsembleConfig
    
    # Create ensemble configuration
    ensemble_config = EnsembleConfig(
        base_models=[
            {'type': 'lstm', 'units': 64, 'dropout': 0.3, 'epochs': 20},
            {'type': 'random_forest', 'n_estimators': 100, 'max_depth': 10}
        ],
        ensemble_method='voting',
        voting_type='soft'
    )
    
    print("Training Deep Learning Ensemble...")
    
    # For ensemble, we'll use flattened data (can handle both types)
    ensemble_model = EnsembleModel(ensemble_config)
    
    ensemble_results = trainer.train_model(
        model=ensemble_model,
        X_train=X_train_flat,
        y_train=y_train_seq,
        X_val=X_val_flat,
        y_val=y_val_seq,
        model_name="dl_ensemble_weather"
    )
    
    print(f"\nEnsemble Training Results:")
    print(f"- Accuracy: {ensemble_results['accuracy']:.3f}")
    print(f"- Precision: {ensemble_results['precision']:.3f}")
    print(f"- Recall: {ensemble_results['recall']:.3f}")
    print(f"- F1 Score: {ensemble_results['f1_score']:.3f}")
    print(f"- Training Time: {ensemble_results['training_time']:.2f}s")
    
    # Test evaluation
    ensemble_test_metrics = ensemble_model.evaluate(X_test_flat, y_test_seq)
    print(f"\nEnsemble Test Results:")
    print(f"- Accuracy: {ensemble_test_metrics.accuracy:.3f}")
    print(f"- Precision: {ensemble_test_metrics.precision:.3f}")
    print(f"- Recall: {ensemble_test_metrics.recall:.3f}")
    print(f"- F1 Score: {ensemble_test_metrics.f1_score:.3f}")
    print(f"- AUC-ROC: {ensemble_test_metrics.auc_roc:.3f}")
    
else:
    print("TensorFlow not available, skipping ensemble training")
    ensemble_model = None
    ensemble_results = None

## 9. Model Comparison and Analysis

Let's compare all our deep learning models.

In [ ]:
# Collect all results for comparison
if TENSORFLOW_AVAILABLE:
    model_results = []
    
    # Add each model if it was trained successfully
    if lstm_model is not None:
        model_results.append({
            'Model': 'LSTM',
            'Accuracy': lstm_test_metrics.accuracy,
            'Precision': lstm_test_metrics.precision,
            'Recall': lstm_test_metrics.recall,
            'F1 Score': lstm_test_metrics.f1_score,
            'AUC-ROC': lstm_test_metrics.auc_roc,
            'Training Time (s)': lstm_results['training_time']
        })
    
    if transformer_model is not None:
        model_results.append({
            'Model': 'Transformer',
            'Accuracy': transformer_test_metrics.accuracy,
            'Precision': transformer_test_metrics.precision,
            'Recall': transformer_test_metrics.recall,
            'F1 Score': transformer_test_metrics.f1_score,
            'AUC-ROC': transformer_test_metrics.auc_roc,
            'Training Time (s)': transformer_results['training_time']
        })
    
    if attention_lstm_model is not None:
        model_results.append({
            'Model': 'Attention-LSTM',
            'Accuracy': attention_lstm_test_metrics.accuracy,
            'Precision': attention_lstm_test_metrics.precision,
            'Recall': attention_lstm_test_metrics.recall,
            'F1 Score': attention_lstm_test_metrics.f1_score,
            'AUC-ROC': attention_lstm_test_metrics.auc_roc,
            'Training Time (s)': attention_lstm_results['training_time']
        })
    
    if cnn_model is not None:
        model_results.append({
            'Model': 'CNN',
            'Accuracy': cnn_test_metrics.accuracy,
            'Precision': cnn_test_metrics.precision,
            'Recall': cnn_test_metrics.recall,
            'F1 Score': cnn_test_metrics.f1_score,
            'AUC-ROC': cnn_test_metrics.auc_roc,
            'Training Time (s)': cnn_results['training_time']
        })
    
    if autoencoder_model is not None:
        model_results.append({
            'Model': 'Autoencoder',
            'Accuracy': autoencoder_test_metrics.accuracy,
            'Precision': autoencoder_test_metrics.precision,
            'Recall': autoencoder_test_metrics.recall,
            'F1 Score': autoencoder_test_metrics.f1_score,
            'AUC-ROC': autoencoder_test_metrics.auc_roc,
            'Training Time (s)': autoencoder_results['training_time']
        })
    
    if ensemble_model is not None:
        model_results.append({
            'Model': 'Ensemble',
            'Accuracy': ensemble_test_metrics.accuracy,
            'Precision': ensemble_test_metrics.precision,
            'Recall': ensemble_test_metrics.recall,
            'F1 Score': ensemble_test_metrics.f1_score,
            'AUC-ROC': ensemble_test_metrics.auc_roc,
            'Training Time (s)': ensemble_results['training_time']
        })
    
    if model_results:
        comparison_df = pd.DataFrame(model_results)
        comparison_df = comparison_df.round(3)
        
        print("Deep Learning Model Comparison:")
        print("=" * 100)
        print(comparison_df.to_string(index=False))
        
        # Find best models
        print("\nBest Models by Metric:")
        print("=" * 50)
        for metric in ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC-ROC']:
            best_idx = comparison_df[metric].idxmax()
            best_model = comparison_df.loc[best_idx, 'Model']
            best_value = comparison_df.loc[best_idx, metric]
            print(f"{metric:>12}: {best_model} ({best_value:.3f})")
    else:
        print("No models were successfully trained for comparison.")

else:
    print("TensorFlow not available, no deep learning models to compare.")

In [ ]:
# Visualize model comparison
if TENSORFLOW_AVAILABLE and 'comparison_df' in locals() and len(comparison_df) > 0:
    # Performance comparison bar chart
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Deep Learning Model Performance Comparison', fontsize=16)
    
    # Accuracy comparison
    axes[0, 0].bar(comparison_df['Model'], comparison_df['Accuracy'], color='skyblue')
    axes[0, 0].set_title('Accuracy Comparison')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].set_ylim(0, 1)
    
    # F1 Score comparison
    axes[0, 1].bar(comparison_df['Model'], comparison_df['F1 Score'], color='lightgreen')
    axes[0, 1].set_title('F1 Score Comparison')
    axes[0, 1].set_ylabel('F1 Score')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].set_ylim(0, 1)
    
    # AUC-ROC comparison
    axes[1, 0].bar(comparison_df['Model'], comparison_df['AUC-ROC'], color='coral')
    axes[1, 0].set_title('AUC-ROC Comparison')
    axes[1, 0].set_ylabel('AUC-ROC')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].set_ylim(0, 1)
    
    # Training time comparison
    axes[1, 1].bar(comparison_df['Model'], comparison_df['Training Time (s)'], color='gold')
    axes[1, 1].set_title('Training Time Comparison')
    axes[1, 1].set_ylabel('Training Time (seconds)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Radar chart for multi-metric comparison
    import math
    
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC-ROC']
    angles = [n / float(len(metrics)) * 2 * math.pi for n in range(len(metrics))]
    angles += angles[:1]  # Complete the circle
    
    fig, ax = plt.subplots(figsize=(12, 10), subplot_kw=dict(projection='polar'))
    
    colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown']
    
    for i, (_, row) in enumerate(comparison_df.iterrows()):
        if i < len(colors):
            values = [row[metric] for metric in metrics]
            values += values[:1]  # Complete the circle
            
            ax.plot(angles, values, 'o-', linewidth=2, label=row['Model'], color=colors[i])
            ax.fill(angles, values, alpha=0.25, color=colors[i])
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(metrics)
    ax.set_ylim(0, 1)
    ax.set_title('Deep Learning Model Performance (Radar Chart)', size=16, y=1.1)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("No comparison data available for visualization.")

## 10. Model Interpretation and Analysis

Let's analyze what our deep learning models have learned.

In [ ]:
# Analyze predictions and patterns
if TENSORFLOW_AVAILABLE and lstm_model is not None:
    # Get predictions from the best model
    lstm_predictions = lstm_model.predict(X_test_seq_scaled)
    lstm_probabilities = lstm_model.predict_proba(X_test_seq_scaled)
    
    # Analyze prediction confidence
    confidence_scores = np.max(lstm_probabilities, axis=1)
    
    # Create prediction analysis
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('LSTM Model Analysis', fontsize=16)
    
    # Prediction confidence distribution
    axes[0, 0].hist(confidence_scores, bins=30, alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Prediction Confidence Distribution')
    axes[0, 0].set_xlabel('Confidence Score')
    axes[0, 0].set_ylabel('Frequency')
    
    # Confidence vs accuracy
    correct_predictions = (lstm_predictions == y_test_seq)
    axes[0, 1].scatter(confidence_scores[correct_predictions], 
                      np.ones(sum(correct_predictions)), 
                      alpha=0.6, label='Correct', s=10)
    axes[0, 1].scatter(confidence_scores[~correct_predictions], 
                      np.zeros(sum(~correct_predictions)), 
                      alpha=0.6, label='Incorrect', s=10, color='red')
    axes[0, 1].set_title('Confidence vs Correctness')
    axes[0, 1].set_xlabel('Confidence Score')
    axes[0, 1].set_ylabel('Correct (1) / Incorrect (0)')
    axes[0, 1].legend()
    
    # Prediction probability distribution
    reg_probs = lstm_probabilities[y_test_seq == 1][:, 1]  # Regulation probability for actual regulations
    no_reg_probs = lstm_probabilities[y_test_seq == 0][:, 1]  # Regulation probability for no regulations
    
    axes[1, 0].hist(no_reg_probs, bins=30, alpha=0.7, label='No Regulation (True)', color='blue')
    axes[1, 0].hist(reg_probs, bins=30, alpha=0.7, label='Regulation (True)', color='red')
    axes[1, 0].set_title('Predicted Probability Distribution')
    axes[1, 0].set_xlabel('Predicted Regulation Probability')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].legend()
    
    # Feature importance proxy (using gradient approximation)
    # For demonstration, we'll show which time steps are most important
    sample_sequences = X_test_seq_scaled[:100]  # Use first 100 test samples
    base_pred = lstm_model.predict_proba(sample_sequences)[:, 1]
    
    # Perturb each time step and measure impact
    time_importance = []
    for t in range(sequence_length):
        perturbed_sequences = sample_sequences.copy()
        perturbed_sequences[:, t, :] = 0  # Zero out time step t
        perturbed_pred = lstm_model.predict_proba(perturbed_sequences)[:, 1]
        importance = np.mean(np.abs(base_pred - perturbed_pred))
        time_importance.append(importance)
    
    axes[1, 1].plot(range(sequence_length), time_importance, marker='o')
    axes[1, 1].set_title('Time Step Importance (Hours Before)')
    axes[1, 1].set_xlabel('Hours Before Current Time')
    axes[1, 1].set_ylabel('Average Impact on Prediction')
    axes[1, 1].invert_xaxis()  # Most recent time on the right
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print insights
    print("Model Analysis Insights:")
    print("=" * 40)
    print(f"Average prediction confidence: {confidence_scores.mean():.3f}")
    print(f"High confidence predictions (>0.9): {(confidence_scores > 0.9).mean():.1%}")
    print(f"Low confidence predictions (<0.6): {(confidence_scores < 0.6).mean():.1%}")
    
    most_important_time = np.argmax(time_importance)
    print(f"Most important time step: {sequence_length - most_important_time} hours before")
    print(f"Recent hours (last 6) average importance: {np.mean(time_importance[-6:]):.4f}")
    print(f"Earlier hours (first 6) average importance: {np.mean(time_importance[:6]):.4f}")

else:
    print("LSTM model not available for analysis.")

## 11. Hyperparameter Tuning for Deep Learning

Let's demonstrate advanced hyperparameter tuning for our best deep learning model.

In [ ]:
if TENSORFLOW_AVAILABLE:
    from training.hyperparameter_tuning import BayesianOptimizationTuner
    
    # Define parameter space for LSTM tuning
    param_space = {
        'units': [32, 128],
        'dropout': [0.1, 0.5],
        'recurrent_dropout': [0.1, 0.5],
        'learning_rate': [0.0001, 0.01],
        'batch_size': [16, 64]
    }
    
    print("Starting Bayesian optimization for LSTM...")
    print(f"Parameter space: {param_space}")
    
    # Create base model for tuning
    base_config = LSTMConfig(epochs=15, sequence_length=sequence_length)  # Fewer epochs for speed
    tuning_model = LSTMModel(base_config)
    
    # Perform Bayesian optimization
    tuner = BayesianOptimizationTuner(n_trials=10)  # Limited trials for demo
    
    tuning_result = tuner.tune(
        model=tuning_model,
        param_space=param_space,
        X_train=X_train_seq_scaled,
        y_train=y_train_seq,
        X_val=X_val_seq_scaled,
        y_val=y_val_seq
    )
    
    print(f"\nBayesian Optimization Results:")
    print(f"Best parameters: {tuning_result.best_params}")
    print(f"Best validation score: {tuning_result.best_score:.3f}")
    print(f"Number of trials: {len(tuning_result.all_results)}")
    
    # Train optimized model
    optimized_config = LSTMConfig(
        **tuning_result.best_params,
        epochs=30,
        sequence_length=sequence_length
    )
    
    optimized_lstm = LSTMModel(optimized_config)
    
    optimized_results = trainer.train_model(
        model=optimized_lstm,
        X_train=X_train_seq_scaled,
        y_train=y_train_seq,
        X_val=X_val_seq_scaled,
        y_val=y_val_seq,
        model_name="optimized_lstm"
    )
    
    # Test optimized model
    optimized_test_metrics = optimized_lstm.evaluate(X_test_seq_scaled, y_test_seq)
    
    print(f"\nOptimized LSTM Test Results:")
    print(f"- Accuracy: {optimized_test_metrics.accuracy:.3f}")
    print(f"- Precision: {optimized_test_metrics.precision:.3f}")
    print(f"- Recall: {optimized_test_metrics.recall:.3f}")
    print(f"- F1 Score: {optimized_test_metrics.f1_score:.3f}")
    print(f"- AUC-ROC: {optimized_test_metrics.auc_roc:.3f}")
    
    # Compare with original
    if lstm_model is not None:
        print(f"\nImprovement over original LSTM:")
        print(f"- Accuracy: {optimized_test_metrics.accuracy - lstm_test_metrics.accuracy:+.3f}")
        print(f"- F1 Score: {optimized_test_metrics.f1_score - lstm_test_metrics.f1_score:+.3f}")
        print(f"- AUC-ROC: {optimized_test_metrics.auc_roc - lstm_test_metrics.auc_roc:+.3f}")
    
else:
    print("TensorFlow not available, skipping hyperparameter tuning.")

## 12. Conclusions and Best Practices

Let's summarize our findings and provide recommendations for deep learning in weather prediction.

In [ ]:
# Summary and recommendations
print("Deep Learning for Weather Regulation Prediction - Summary")
print("=" * 70)

if TENSORFLOW_AVAILABLE and 'comparison_df' in locals() and len(comparison_df) > 0:
    # Find best overall model
    best_f1_idx = comparison_df['F1 Score'].idxmax()
    best_model = comparison_df.loc[best_f1_idx, 'Model']
    best_f1 = comparison_df.loc[best_f1_idx, 'F1 Score']
    
    print(f"\n🏆 Best Overall Model: {best_model} (F1: {best_f1:.3f})")
    
    # Model-specific insights
    print("\n📊 Model-Specific Insights:")
    for _, row in comparison_df.iterrows():
        model = row['Model']
        if model == 'LSTM':
            print(f"   • {model}: Good for sequential patterns, moderate training time")
        elif model == 'Transformer':
            print(f"   • {model}: State-of-the-art attention, longer training time")
        elif model == 'Attention-LSTM':
            print(f"   • {model}: Combines LSTM memory with attention focus")
        elif model == 'CNN':
            print(f"   • {model}: Captures local patterns, fast training")
        elif model == 'Autoencoder':
            print(f"   • {model}: Learns compressed representations, good for feature learning")
        elif model == 'Ensemble':
            print(f"   • {model}: Combines strengths of multiple models, robust predictions")
    
    print("\n🎯 Key Findings:")
    avg_accuracy = comparison_df['Accuracy'].mean()
    avg_f1 = comparison_df['F1 Score'].mean()
    print(f"   • Average deep learning accuracy: {avg_accuracy:.3f}")
    print(f"   • Average F1 score: {avg_f1:.3f}")
    
    fastest_model = comparison_df.loc[comparison_df['Training Time (s)'].idxmin(), 'Model']
    slowest_model = comparison_df.loc[comparison_df['Training Time (s)'].idxmax(), 'Model']
    print(f"   • Fastest training: {fastest_model}")
    print(f"   • Slowest training: {slowest_model}")

print("\n🔬 Technical Insights:")
print("   • Sequential models (LSTM, Transformer) excel at temporal patterns")
print("   • Attention mechanisms help focus on critical time periods")
print("   • Ensemble methods provide robust, well-calibrated predictions")
print("   • CNNs can capture spatial-temporal patterns when data is reshaped")
print("   • Autoencoders useful for dimensionality reduction and anomaly detection")

print("\n⚡ Performance Optimization Tips:")
print("   • Use bidirectional LSTM for better context understanding")
print("   • Apply dropout (0.2-0.4) to prevent overfitting")
print("   • Sequence length of 12-48 hours usually optimal for weather")
print("   • Batch size 32-64 provides good speed/memory trade-off")
print("   • Early stopping prevents overtraining")
print("   • Learning rate scheduling improves convergence")

print("\n🎛️ Hyperparameter Recommendations:")
print("   • LSTM units: 32-128 (balance complexity vs overfitting)")
print("   • Transformer heads: 4-8 (match data complexity)")
print("   • Learning rates: 0.001-0.01 (start conservative)")
print("   • Use Bayesian optimization for efficient search")
print("   • Monitor validation metrics to avoid overfitting")

print("\n🚀 Next Steps for Production:")
print("   • Implement real-time inference pipeline")
print("   • Add model monitoring and drift detection")
print("   • Set up automated retraining workflows")
print("   • Create ensemble of best-performing models")
print("   • Implement A/B testing for model comparison")
print("   • Add uncertainty quantification for predictions")

print("\n📈 Future Enhancements:")
print("   • Multi-modal fusion (weather radar + satellite imagery)")
print("   • Graph neural networks for airport network effects")
print("   • Federated learning across multiple airports")
print("   • Explainable AI for regulatory compliance")
print("   • Real-time model updates with streaming data")

print("\n" + "=" * 70)
print("Thank you for exploring deep learning for weather regulation prediction!")
print("For more advanced techniques, check out the other tutorial notebooks.")

## Deep Learning with Balanced vs Imbalanced Data

Let's demonstrate the critical importance of balanced datasets for deep learning models by comparing the performance with imbalanced training. This comparison reveals why balanced datasets are essential for aviation regulation prediction.

In [ ]:
# Demonstrate impact of balanced vs imbalanced training on deep learning
if TENSORFLOW_AVAILABLE and use_balanced and lstm_model is not None:
    print("🎯 Deep Learning: Balanced vs Imbalanced Training Comparison")
    print("=" * 70)
    
    # Create imbalanced training set (simulate real-world 8% regulation rate)
    n_samples = len(X_train_seq_scaled)
    regulation_rate = 0.08
    n_regulations = int(regulation_rate * n_samples)
    n_no_regulations = n_samples - n_regulations
    
    # Get balanced indices
    reg_indices = np.where(y_train_seq == 1)[0]
    no_reg_indices = np.where(y_train_seq == 0)[0]
    
    # Create imbalanced subset
    imbalanced_reg_idx = reg_indices[:n_regulations]
    imbalanced_no_reg_idx = no_reg_indices[:n_no_regulations]
    imbalanced_indices = np.concatenate([imbalanced_reg_idx, imbalanced_no_reg_idx])
    
    X_train_imbalanced = X_train_seq_scaled[imbalanced_indices]
    y_train_imbalanced = y_train_seq[imbalanced_indices]
    
    print(f"Imbalanced training set:")
    print(f"  • Total samples: {len(y_train_imbalanced)}")
    print(f"  • Regulations: {y_train_imbalanced.sum()} ({y_train_imbalanced.mean():.1%})")
    print(f"  • No regulations: {len(y_train_imbalanced) - y_train_imbalanced.sum()}")
    
    # Train LSTM on imbalanced data
    print(f"\n🔄 Training LSTM on IMBALANCED dataset...")
    lstm_imbalanced_config = LSTMConfig(
        units=64,
        dropout=0.3,
        recurrent_dropout=0.3,
        bidirectional=True,
        batch_size=32,
        epochs=30,
        sequence_length=sequence_length,
        learning_rate=0.001,
        early_stopping_patience=8
    )
    
    lstm_imbalanced = LSTMModel(lstm_imbalanced_config)
    
    lstm_imbalanced_results = trainer.train_model(
        model=lstm_imbalanced,
        X_train=X_train_imbalanced,
        y_train=y_train_imbalanced,
        X_val=X_val_seq_scaled,
        y_val=y_val_seq,
        model_name="lstm_imbalanced"
    )
    
    # Evaluate both models on test set
    lstm_balanced_metrics = lstm_model.evaluate(X_test_seq_scaled, y_test_seq)
    lstm_imbalanced_metrics = lstm_imbalanced.evaluate(X_test_seq_scaled, y_test_seq)
    
    # Comprehensive comparison
    print(f"\n📊 LSTM: Balanced vs Imbalanced Training Results")
    print("=" * 80)
    print(f"{'Metric':<20} {'Balanced':<15} {'Imbalanced':<15} {'Improvement':<15} {'Relative Gain':<15}")
    print("-" * 80)
    
    metrics_data = [
        ('Accuracy', lstm_balanced_metrics.accuracy, lstm_imbalanced_metrics.accuracy),
        ('Precision', lstm_balanced_metrics.precision, lstm_imbalanced_metrics.precision),
        ('Recall', lstm_balanced_metrics.recall, lstm_imbalanced_metrics.recall),
        ('F1 Score', lstm_balanced_metrics.f1_score, lstm_imbalanced_metrics.f1_score),
        ('AUC-ROC', lstm_balanced_metrics.auc_roc, lstm_imbalanced_metrics.auc_roc)
    ]
    
    improvements = []
    for metric_name, balanced_val, imbalanced_val in metrics_data:
        abs_improvement = balanced_val - imbalanced_val
        rel_improvement = (abs_improvement / imbalanced_val) * 100 if imbalanced_val > 0 else 0
        improvements.append((metric_name, abs_improvement, rel_improvement))
        
        print(f"{metric_name:<20} {balanced_val:<15.3f} {imbalanced_val:<15.3f} {abs_improvement:<15.3f} {rel_improvement:<14.1f}%")
    
    # Highlight key insights
    print(f"\n💡 Key Deep Learning Insights:")
    recall_improvement = next(imp for metric, imp, _ in improvements if metric == 'Recall')[1]
    f1_improvement = next(imp for metric, imp, _ in improvements if metric == 'F1 Score')[1]
    
    print(f"  • Recall improvement: +{recall_improvement:.3f} (Critical for detecting regulations!)")
    print(f"  • F1 Score improvement: +{f1_improvement:.3f} (Better overall performance)")
    print(f"  • Balanced training helps LSTM learn meaningful sequential patterns")
    print(f"  • Neural networks are particularly sensitive to class imbalance")
    
    # Visualize prediction distributions
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Deep Learning: Balanced vs Imbalanced Training Comparison', fontsize=16)
    
    # Get predictions
    balanced_probs = lstm_model.predict_proba(X_test_seq_scaled)[:, 1]
    imbalanced_probs = lstm_imbalanced.predict_proba(X_test_seq_scaled)[:, 1]
    
    # Prediction probability distributions
    axes[0, 0].hist(balanced_probs[y_test_seq == 0], bins=30, alpha=0.7, label='No Regulation', color='blue')
    axes[0, 0].hist(balanced_probs[y_test_seq == 1], bins=30, alpha=0.7, label='Regulation', color='red')
    axes[0, 0].set_title('Balanced LSTM - Prediction Probabilities')
    axes[0, 0].set_xlabel('Predicted Probability of Regulation')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].legend()
    
    axes[0, 1].hist(imbalanced_probs[y_test_seq == 0], bins=30, alpha=0.7, label='No Regulation', color='blue')
    axes[0, 1].hist(imbalanced_probs[y_test_seq == 1], bins=30, alpha=0.7, label='Regulation', color='red')
    axes[0, 1].set_title('Imbalanced LSTM - Prediction Probabilities')
    axes[0, 1].set_xlabel('Predicted Probability of Regulation')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].legend()
    
    # Confusion matrices
    from sklearn.metrics import confusion_matrix
    
    balanced_preds = lstm_model.predict(X_test_seq_scaled)
    imbalanced_preds = lstm_imbalanced.predict(X_test_seq_scaled)
    
    cm_balanced = confusion_matrix(y_test_seq, balanced_preds)
    cm_imbalanced = confusion_matrix(y_test_seq, imbalanced_preds)
    
    sns.heatmap(cm_balanced, annot=True, fmt='d', cmap='Blues', ax=axes[1, 0])
    axes[1, 0].set_title('Balanced LSTM - Confusion Matrix')
    axes[1, 0].set_xlabel('Predicted')
    axes[1, 0].set_ylabel('Actual')
    
    sns.heatmap(cm_imbalanced, annot=True, fmt='d', cmap='Reds', ax=axes[1, 1])
    axes[1, 1].set_title('Imbalanced LSTM - Confusion Matrix')
    axes[1, 1].set_xlabel('Predicted')
    axes[1, 1].set_ylabel('Actual')
    
    plt.tight_layout()
    plt.show()
    
    # Learning curve comparison (if training history available)
    if hasattr(lstm_model, 'training_history') and hasattr(lstm_imbalanced, 'training_history'):
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Loss comparison
        if lstm_model.training_history and 'loss' in lstm_model.training_history:
            axes[0].plot(lstm_model.training_history['loss'], label='Balanced Training Loss', color='blue')
            axes[0].plot(lstm_model.training_history.get('val_loss', []), label='Balanced Validation Loss', color='lightblue')
        
        if lstm_imbalanced.training_history and 'loss' in lstm_imbalanced.training_history:
            axes[0].plot(lstm_imbalanced.training_history['loss'], label='Imbalanced Training Loss', color='red')
            axes[0].plot(lstm_imbalanced.training_history.get('val_loss', []), label='Imbalanced Validation Loss', color='lightcoral')
        
        axes[0].set_title('Training Loss Comparison')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Accuracy comparison
        if lstm_model.training_history and 'accuracy' in lstm_model.training_history:
            axes[1].plot(lstm_model.training_history['accuracy'], label='Balanced Training Accuracy', color='blue')
            axes[1].plot(lstm_model.training_history.get('val_accuracy', []), label='Balanced Validation Accuracy', color='lightblue')
        
        if lstm_imbalanced.training_history and 'accuracy' in lstm_imbalanced.training_history:
            axes[1].plot(lstm_imbalanced.training_history['accuracy'], label='Imbalanced Training Accuracy', color='red')
            axes[1].plot(lstm_imbalanced.training_history.get('val_accuracy', []), label='Imbalanced Validation Accuracy', color='lightcoral')
        
        axes[1].set_title('Training Accuracy Comparison')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Accuracy')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print(f"\n🎯 Conclusion:")
    print(f"Balanced datasets are CRITICAL for deep learning in aviation regulation prediction:")
    print(f"• Enable better detection of rare but important regulation events")
    print(f"• Improve gradient flow and feature learning in neural networks") 
    print(f"• Provide more reliable and calibrated probability predictions")
    print(f"• Essential for operational deployment where missing regulations is costly")
    
elif not use_balanced:
    print("This demonstration requires the balanced dataset.")
    print("The benefits of balanced training for deep learning include:")
    print("• Better gradient flow during backpropagation")
    print("• Improved feature learning for both classes")
    print("• More reliable probability calibration")
    print("• Enhanced pattern recognition capabilities")
    
elif not TENSORFLOW_AVAILABLE:
    print("TensorFlow is required for this deep learning comparison.")
    
else:
    print("LSTM model training is required for this comparison.")